In [10]:
import hashlib
import requests

In [11]:
password = '12345Qwert'
salt = 'whatAnAmazingCon'

# MD5 Hashes (Vulnerable)

In [12]:
def get_md5_hash(text: str) -> str:
    md5_hash = hashlib.md5()
    md5_hash.update(text.encode('utf-8'))
    hash_result = md5_hash.hexdigest()

    return hash_result

def get_md5_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text

    return get_md5_hash(salted_text)



In [13]:
hash = get_md5_hash(password)
salted_hash = get_md5_salted_hash(password, salt)

print(f'The md5 hash is: {hash}')
print(f'The salted md5 hash is: {salted_hash}')

The md5 hash is: 18a49c364ba10330d894b11652e65bc2
The salted md5 hash is: ec31aadc6a06fe53ca079564c48596a6


# SHA 256 Hashes

In [14]:
def get_sha256_hash(text: str) -> str:
    return hashlib.sha256(text.encode()).hexdigest()

def get_sha256_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text
    return get_sha256_hash(salted_text)

In [15]:
hash = get_sha256_hash(password)
salted_hash = get_sha256_salted_hash(password, salt)

print(f'The sha256 hash is: {hash}')
print(f'The salted sha256 hash is: {salted_hash}')

The sha256 hash is: c90142696762be2f4632c18382315ab39e20a41e997cd8a41502aeafcfae49fe
The salted sha256 hash is: b60c0049182cc396182f62cf598c39a4cfe73ce889f31c840014c09cdc1b3bb6


# Certificates Checker


In [16]:
pip install certvalidator


Note: you may need to restart the kernel to use updated packages.


In [18]:
from oscrypto import tls
from certvalidator import CertificateValidator


expired_cert = 'expired.badssl.com'
self_signed = 'self-signed.badssl.com'
bad_dh = 'dh480.badssl.com'

try:
    session = tls.TLSSession(manual_validation=True)
    connection = tls.TLSSocket(expired_cert, 443, session=session)

    validator = CertificateValidator(connection.certificate, connection.intermediates)
    validator.validate_tls(connection.hostname)
except Exception as err:
    print(f"Unexpected {err}")
    print(f" type: {type(err)}")

Unexpected The path could not be validated because the end-entity certificate expired 2015-04-12 23:59:59Z
 type: <class 'certvalidator.errors.PathValidationError'>


# Check for filtered password using HIBP (Have I Been Pwned?)

In [19]:
password = '1234'

In [20]:
base_url = "https://api.pwnedpasswords.com/range"
# Process obtained from https://www.troyhunt.com/ive-just-launched-pwned-passwords-version-2/#cloudflareprivacyandkanonymity
# First we obtain the first 5 characters of the password encoded in SHA-1
sha1_hash = hashlib.sha1(password.encode()).hexdigest()
range = sha1_hash[:5].upper()

# We obtain all filtered passwords in that range and check for ours (with anonimity, there will be about 475 results, we are NOT giving away our password)
r = requests.get(f'{base_url}/{range}')
response_text = r.text
index = response_text.find(sha1_hash[5:].upper())

if index != -1:
  leaked_number = response_text[index:].split('\r\n')[0].split(':')[1]
  print(f'This password has been seen {leaked_number} times before...')
else:
  print('This password has not been leaked (yet...)')


This password has been seen 1473205 times before...


# References
1. [Salt OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#salting)
1. [Pepper OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#peppering)
